<h3>Abstract model</h3>

An abstract formulation is useful for managing different instances of a problem, provided its general structure remains always the same.
This section shows how to create an abstract model formulated as follows:

$\max{z} = \sum_{j=1}^{n} c_jx_j$
<br>
<br>
Subject to:
<br>
<br>
$\sum_{j=1}^{n} a_{ij}x_j \leq b_i, \forall i = 1 \dotsc m$
<br>
$x_j \geq d_j, \forall j = 1 \dotsc n$
<br>
$x_j \in \Z_{+}, \forall j = 1 \dotsc n$
<br>
<br>
The chosen solver is CPLEX by IBM.

In [ ]:
from __future__ import division
import pyomo.environ as pyo

model = pyo.AbstractModel()

model.m = pyo.Param(within=pyo.NonNegativeIntegers)
model.n = pyo.Param(within=pyo.NonNegativeIntegers)

model.i = pyo.RangeSet(1, model.m)
model.j = pyo.RangeSet(1, model.n)

model.a = pyo.Param(model.i, model.j)
model.b = pyo.Param(model.i)
model.c = pyo.Param(model.j)
model.d = pyo.Param(model.j)

# used to define bounds on variables
def bounds_rule(model, j):
    return (model.d[j], None)

model.x = pyo.Var(model.j, domain=pyo.NonNegativeIntegers, bounds=bounds_rule)

# describes the objective function
def obj_expression(model):
    return pyo.summation(model.c, model.x)

model.obj = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

# describes constraints of the form Ax <= b
def axb_constraint_rule(model, i):
    return sum(model.a[i,j] * model.x[j] for j in model.j) <= model.b[i]

model.axb = pyo.Constraint(model.i, rule=axb_constraint_rule)

# creates an instance based on data specified in "data.dat" file
instance = model.create_instance(filename="data.dat")

# outputs a summary of the model's instance
instance.pprint()

Now we can pass our model's instance to a solver (CPLEX in this case) to get a result:

In [ ]:
solver = pyo.SolverFactory('cplex')
result = solver.solve(instance)

# show results for the solved model's instance
instance.display()